📌 새 고객데이터를 pkl 모델에 덮어서 이탈율 예측하기

모델 불러오기

이미 만들어둔 pipeline_customer_churn_model.pkl

새 고객 데이터 불러오기

testpurpose.csv

이탈 확률 계산

model.predict_proba() 사용

세그먼트 분류

High Risk, High Value 등 비즈니스 규칙 적용

새 CSV로 저장

segmented_testpurpose.csv

📌 규칙 기반 vs 클러스터링 기반 차이

규칙 기반 (if/else)

사람이 직접 기준을 정함 (예: ChurnProbability ≥ 0.5 → High Risk).

장점: 해석 쉬움, 비즈니스팀이 이해하기 좋음.

단점: 기준값이 임의적임 (0.5? 0.3? 80달러?).

클러스터링 기반 (KMeans 등)

머신러닝이 데이터를 보고 자동으로 “비슷한 고객끼리 묶음”.

장점: 패턴을 스스로 찾아냄 → 규칙으로 잡기 힘든 그룹도 발견 가능.

단점: 각 그룹이 왜 그렇게 나뉘었는지 해석은 어려울 수 있음.

In [3]:
import pickle
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# ---------------------------
# 0. 학습 파이프라인에서 쓰던 clean_numeric 함수 정의
# ---------------------------
def clean_numeric(X):
    X = X.copy()
    for col in ["TotalCharges", "MonthlyCharges", "tenure"]:
        X[col] = pd.to_numeric(X[col], errors="coerce").fillna(0)
    return X



# ---------------------------
# 1. 저장된 모델 불러오기
# ---------------------------
# pipeline_customer_churn_model.pkl 안에는
#   - 전처리 (OneHotEncoder 등)
#   - SMOTE (훈련용 오버샘플링)
#   - RandomForestClassifier (학습된 모델)
#   - 확률 보정(Calibration)
# 이 모두가 포함되어 있음
with open(r"C:\Users\honor\spicedAcademy\Capstone_Final_Project\Retain_Flow_Automation-\notebook\pipeline_customer_churn_model.pkl", "rb") as f:
    model = pickle.load(f)

# ---------------------------
# 2. 새로운 고객 데이터 불러오기
# ---------------------------
# 이 파일은 아직 전처리 안 된 "원본 고객 CSV"
# 파이프라인 안에서 자동으로 전처리가 되므로
# 그냥 원본 그대로 불러오면 됨
df_new = pd.read_csv("TestPurpose.csv")

# ---------------------------
# 3. 이탈 확률 예측
# ---------------------------
# predict_proba → 확률 예측 메소드
#   model.predict_proba(X)[:, 1] 은
#   "이탈할 확률" (클래스 1의 확률)만 추출
df_new["ChurnProbability"] = model.predict_proba(df_new)[:, 1]

# ---------------------------
# 4. 클러스터링 준비
# ---------------------------
# KMeans는 숫자 값 크기에 민감하기 때문에 스케일링 필요
X_cluster = df_new[["ChurnProbability", "MonthlyCharges"]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)
# 👉 클러스터링에 쓸 변수는 이탈 확률 + 월 요금.
# 이 두 값은 범위가 다르기 때문에(0~1 vs 20~120) 표준화(StandardScaler)로 크기를 맞춥니다.

# ---------------------------
# 5. KMeans 실행 (예: 4개 그룹)
# ---------------------------
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_new["Cluster"] = kmeans.fit_predict(X_scaled)
# n_clusters=4 → 고객을 4개 그룹으로 나눔.
# fit_predict() → 각 고객이 어느 그룹에 속하는지 번호(0,1,2,3) 붙임.


# ---------------------------
# 6. 각 그룹 해석 (Cluster → 라벨 매핑)
# ---------------------------
def label_cluster(row):
    if row["Cluster"] == 2:
        return "High Risk & High Value"
    elif row["Cluster"] == 0:
        return "Low Risk & High Value"
    elif row["Cluster"] == 1:
        return "Low Risk & Low Value"
    elif row["Cluster"] == 3:
        return "Low Risk & Mid Value"
    
# Cluster 번호(0,1,2,3)은 의미가 없으니 주의!!
#   평균 이탈확률 높음, 월요금도 높음 → High Risk & High Value
#   평균 이탈확률 낮음, 월요금 높음 → Low Risk & High Value
#   평균 이탈확률 낮음, 월요금 낮음 → Low Risk & Low Value
#   평균 이탈확률 낮음, 월요금 중간 → Low Risk & Mid Value (임시 라벨)

# KMeans는 항상 데이터를 보고 그룹을 나누는 방식이기 때문에,
# 데이터셋마다 Cluster 0,1,2,3에 속하는 고객들의 평균값이 달라집니다.

df_new["ClusterLabel"] = df_new.apply(label_cluster, axis=1)


# ---------------------------
# 6. 각 그룹 특징 보기
# ---------------------------
print(df_new.groupby("Cluster")[["ChurnProbability", "MonthlyCharges"]].mean())
# 👉 각 그룹별 평균 이탈 확률 / 월 요금 출력 → 그룹 해석의 핵심!
# ---------------------------
# 7. 결과 저장
# ---------------------------
df_new.to_csv("segmented_testpurpose_kmeans.csv", index=False)
print("✅ segmented_testpurpose_kmeans.csv 생성 완료")

         ChurnProbability  MonthlyCharges
Cluster                                  
0                0.158166       96.645952
1                0.117386       23.095275
2                0.634630       78.105246
3                0.138676       61.209551
✅ segmented_testpurpose_kmeans.csv 생성 완료


📌 결과 해석 예시

👉 규칙 기반처럼 네 그룹이 나오는데, 이번엔 사람이 기준을 정한 게 아니라 알고리즘이 데이터 패턴을 보고 자동으로 묶은 것이에요.

print(df_new.groupby(...).mean()) 결과가 이렇게 나올 수 있어요:

Cluster	평균 ChurnProbability	평균 MonthlyCharges	해석

0	0.75	95.0	High Risk & High Value

1	0.70	40.0	High Risk & Low Value

2	0.20	100.0	Low Risk & High Value

3	0.10	30.0	Low Risk & Low Value



📌 정리

규칙 기반: 사람이 cutoff 값을 정한다 (해석 쉬움).

클러스터링 기반: 머신이 그룹을 찾는다 (데이터 패턴에 맞음).

결과적으로 고객을 4개 그룹으로 나눠서, 비즈니스 전략을 다르게 적용할 수 있음.